In [1]:
"""
Notebook basics.

This notebook shows you the basic steps for loading and displaying data
from checklists and observations. It assumes you have completed he steps in
the "Getting Started" section of the README, and you have an eBird API key.
"""
from django.conf import settings
from django.template.loader import render_to_string

from faker import Faker
from IPython.display import display, HTML

from ebird.checklists.loaders import APILoader, SpeciesLoader
from ebird.checklists.models import Checklist, Observation

# Randomly choose a country to load checklists for.
region = Faker().country_code()

In [2]:
# Load the complete eBird taxonomy into the Species table
loader = SpeciesLoader(settings.EBIRD_API_KEY)
loader.load()

2025-01-08 22:59 [INFO] Loading eBird taxonomy
2025-01-08 23:03 [INFO] Loaded eBird taxonomy


In [4]:
# Load the ten most recent checklists from the eBird API.
loader = APILoader(settings.EBIRD_API_KEY)
visits = loader.get_visits(region)[:10]
for visit in visits:
    loader.create_or_update_checklist(visit)

2025-01-08 23:07 [INFO] eBird API: checklists submitted: 200
2025-01-08 23:07 [INFO] eBird API: loading checklist: S208354067
2025-01-08 23:07 [INFO] eBird API: loading checklist: S207790576
2025-01-08 23:07 [INFO] eBird API: loading checklist: S207192511
2025-01-08 23:07 [INFO] eBird API: loading checklist: S207319993
2025-01-08 23:07 [INFO] eBird API: loading checklist: S208110843
2025-01-08 23:07 [INFO] eBird API: loading checklist: S207637949
2025-01-08 23:07 [INFO] eBird API: loading checklist: S207048521
2025-01-08 23:07 [INFO] eBird API: loading checklist: S207048518
2025-01-08 23:07 [INFO] eBird API: loading checklist: S207048516
2025-01-08 23:07 [INFO] eBird API: loading checklist: S207048628


In [5]:
# Show all the checklists just loaded.
checklists = Checklist.objects.filter(location__country_code=region)

table = render_to_string("checklists.html", {"checklists": checklists})
display(HTML(table))

Identifier,Date,Time,Location,Region,Country
S208354067,"Jan. 7, 2025",10:15 a.m.,ADA University Campus,Baki,Azerbaijan
S207790576,"Jan. 3, 2025",10:37 a.m.,"Auto selected 39.72338, 49.17065",Salyan,Azerbaijan
S207192511,"Dec. 30, 2024",3:39 p.m.,"Auto selected 40.32284, 50.31216",Baki,Azerbaijan
S207319993,"Dec. 30, 2024",2:08 p.m.,"Auto selected 40.32284, 50.31216",Baki,Azerbaijan
S208110843,"Dec. 30, 2024",11:13 a.m.,Absheron National Park—main buildings,Baki,Azerbaijan
S207637949,"Dec. 30, 2024",9:02 a.m.,Absheron Park--General Area,Baki,Azerbaijan
S207048521,"Dec. 29, 2024",3:05 p.m.,Absheron Park--General Area,Baki,Azerbaijan
S207048518,"Dec. 29, 2024",3:05 p.m.,Absheron Park--General Area,Baki,Azerbaijan
S207048516,"Dec. 29, 2024",3:05 p.m.,Absheron Park--General Area,Baki,Azerbaijan
S207048628,"Dec. 29, 2024",10:25 a.m.,Cape Gilazi,Xizi,Azerbaijan


In [6]:
# Show the highest count for each species seen
# Rather than build a complex query to group the observations by species
# and calculate the highest count, we cheat by using the {% ifchanged %}
# tag in the template to display the observation each time the species
# changes. Sorting by count descending means this record is the one where
# the highest count was recorded.
observations = Observation.objects.filter(location__country_code=region).order_by("species__taxon_order", "-count")

table = render_to_string("highest-counts.html", {"observations": observations})
display(HTML(table))

Count,Species,Date,Time,Location,Region,Country
20,Mute Swan (Cygnus olor),"Dec. 29, 2024",3:05 p.m.,Absheron Park--General Area,Baki,Azerbaijan
20,Tundra Swan (Cygnus columbianus),"Dec. 30, 2024",9:02 a.m.,Absheron Park--General Area,Baki,Azerbaijan
10,Whooper Swan (Cygnus cygnus),"Jan. 3, 2025",10:37 a.m.,"Auto selected 39.72338, 49.17065",Salyan,Azerbaijan
50,swan sp. (Cygnus sp.),"Dec. 29, 2024",3:05 p.m.,Absheron Park--General Area,Baki,Azerbaijan
18,goose sp. (Anatidae (goose sp.)),"Jan. 3, 2025",10:37 a.m.,"Auto selected 39.72338, 49.17065",Salyan,Azerbaijan
6,Ruddy Shelduck (Tadorna ferruginea),"Jan. 3, 2025",10:37 a.m.,"Auto selected 39.72338, 49.17065",Salyan,Azerbaijan
60,Common Shelduck (Tadorna tadorna),"Jan. 3, 2025",10:37 a.m.,"Auto selected 39.72338, 49.17065",Salyan,Azerbaijan
70,Northern Shoveler (Spatula clypeata),"Jan. 3, 2025",10:37 a.m.,"Auto selected 39.72338, 49.17065",Salyan,Azerbaijan
200,Gadwall (Mareca strepera),"Dec. 30, 2024",2:08 p.m.,"Auto selected 40.32284, 50.31216",Baki,Azerbaijan
500,Eurasian Wigeon (Mareca penelope),"Dec. 29, 2024",3:05 p.m.,Absheron Park--General Area,Baki,Azerbaijan
